In [ ]:
# Ali Khadangi
# Atiye Bonakdar

In [ ]:
!gdown '1Ta0xmh5rRln7lVcYgiQ_1UjN0Xl6VEVM'
!gdown '1E0fhBRhp0hzSlwR2VOchFap6pi52xRDr'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Downloading...
From (original): https://drive.google.com/uc?id=1Ta0xmh5rRln7lVcYgiQ_1UjN0Xl6VEVM
From (redirected): https://drive.google.com/uc?id=1Ta0xmh5rRln7lVcYgiQ_1UjN0Xl6VEVM&confirm=t&uuid=a95ebedb-aa92-4010-aade-9cebe67d185e
To: /content/Reference-Dataset.zip
100% 42.2M/42.2M [00:00<00:00, 85.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1E0fhBRhp0hzSlwR2VOchFap6pi52xRDr
To: /content/GooglePlay.csv
100% 1.32M/1.32M [00:00<00:00, 28.2MB/s]


In [ ]:
!unzip Reference-Dataset.zip

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Archive:  Reference-Dataset.zip
replace Reference-Dataset/Playstore_final.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
import warnings
from sklearn.impute import SimpleImputer
import numpy as np

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 50)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data = pd.read_csv('Reference-Dataset/Playstore_final.csv')
data2 = pd.read_csv('GooglePlay.csv')

In [ ]:
# merge two datasets together
merged_data = pd.merge(data2.rename(columns={"App": "App Name"}), data, on="App Name", how="inner")
merged_data = merged_data.drop(columns=[col for col in merged_data.columns if col.startswith('Unnamed')])

In [ ]:
# match similar columns in each row of dataset and finding inconsistencies
match_columns = {
        'Category_x': 'Category_y',
        'Rating_x': 'Rating_y',
        'Reviews_x': 'Reviews_y',
        'Size_x': 'Size_y',
        'Installs_x': 'Installs_y',
        'Type': 'Free',
        'Price_x': 'Price_y',
        'Content Rating_x': 'Content Rating_y',
        'Genres': 'Category_y',
        'Last Updated': 'Last update',
        'Current Ver': 'Version',
        'Android Ver': 'Android version Text'
    }

inconsistencies = []

for index, row in merged_data.iterrows():
    for col in merged_data.columns:
        if col in match_columns.keys():
            if col == 'Type':
                value1 = merged_data.at[index, col]
                value2 = merged_data.at[index, match_columns[col]]
                if (value1 == 'Free' and value2 == False) or (value1 == 'Paid' and value2 == True):
                    inconsistencies.append('app: ' + row['App Name'] + ' row: ' + str(index) + ' column: ' + col)
            else:
                if merged_data.at[index, col] != merged_data.at[index, match_columns[col]]:
                    inconsistencies.append('app: ' + row['App Name'] + ' row: ' + str(index) + ' column: ' + col)
                merged_data.at[index, col] = merged_data.at[index, match_columns[col]]

In [ ]:
inconsistencies

['app: Smoke Effect Photo Maker - Smoke Editor row: 0 column: Category_x',
 'app: Smoke Effect Photo Maker - Smoke Editor row: 0 column: Rating_x',
 'app: Smoke Effect Photo Maker - Smoke Editor row: 0 column: Reviews_x',
 'app: Smoke Effect Photo Maker - Smoke Editor row: 0 column: Installs_x',
 'app: Smoke Effect Photo Maker - Smoke Editor row: 0 column: Price_x',
 'app: Infinite Painter row: 1 column: Category_x',
 'app: Infinite Painter row: 1 column: Rating_x',
 'app: Infinite Painter row: 1 column: Reviews_x',
 'app: Infinite Painter row: 1 column: Size_x',
 'app: Infinite Painter row: 1 column: Installs_x',
 'app: Infinite Painter row: 1 column: Price_x',
 'app: Infinite Painter row: 1 column: Last Updated',
 'app: Infinite Painter row: 1 column: Current Ver',
 'app: Infinite Painter row: 1 column: Android Ver',
 'app: Garden Coloring Book row: 2 column: Category_x',
 'app: Garden Coloring Book row: 2 column: Rating_x',
 'app: Garden Coloring Book row: 2 column: Reviews_x',
 'ap

In [ ]:
new_column_names = {
    'Category_x': 'Category',
    'Rating_x': 'Rating',
    'Reviews_x': 'Reviews',
    'Size_x': 'Size',
    'Price_x': 'Price',
    'Installs_x': 'Installs',
    'Content Rating_x': 'Content Rating'
}

merged_data = merged_data.rename(columns=new_column_names)
merged_data = merged_data.drop(columns=[col for col in merged_data.columns if col in match_columns.values()])

# combine columns together
prices_asString = merged_data['Price'].astype(str)
merged_data['price_currency'] = prices_asString.str.cat(merged_data['Currency'], sep=" ")

rating_asString = merged_data['Rating'].astype(str)
rating_count_asString = merged_data['Rating Count'].astype(str)
merged_data['rating_and_count'] = rating_asString.str.cat(rating_count_asString, sep=" - ")

merged_data['name_and_category'] = merged_data['App Name'].str.cat(merged_data['Category'], sep=" - ")

editor_choice_asString = merged_data['Editor Choice'].astype(str)
installs_asString = merged_data['Installs'].astype(str)
merged_data['installs_and_editor_choice'] = installs_asString.str.cat(editor_choice_asString, sep=" - ")

In [ ]:
# handle missing values
numeric_columns = [
        'Rating', 'Rating Count', 'Minimum Installs', 'Price', 'Reviews'
    ]

mean_imputer = SimpleImputer(strategy='mean')
for col in numeric_columns:
    merged_data[col] = mean_imputer.fit_transform(merged_data[[col]])

In [ ]:
merged_data.columns

Index(['App Name', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver', 'App Id', 'Rating Count', 'Minimum Installs', 'Currency',
       'Minimum Android', 'Developer Id', 'Developer Website',
       'Developer Email', 'Released', 'Privacy Policy', 'Ad Supported',
       'In app purchases', 'Editor Choice', 'Summary', 'Developer',
       'Developer Address', 'Developer Internal ID', 'price_currency',
       'rating_and_count', 'name_and_category', 'installs_and_editor_choice'],
      dtype='object')

In [ ]:
categorical_columns = [
    'Category', 'Type', 'Content Rating', 'Genres', 'Android Ver', 'Currency', 'Minimum Android', 'Ad Supported', 'In app purchases', 'Editor Choice',
]

for col in categorical_columns:
    mode_value = merged_data[col].mode().iloc[0]
    merged_data[col].fillna(mode_value, inplace=True)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

normalized_data = merged_data.copy()
normalization_columns = ['Category', 'Type', 'Content Rating', 'Genres', 'Android Ver', 'Currency', 'Minimum Android', 'Ad Supported', 'In app purchases', 'Editor Choice']
encoder = OneHotEncoder(sparse=False)
for col in normalization_columns:

  normalized_data[col] = encoder.fit_transform(merged_data[[col]])
print(normalized_data.head(50))

                                             App Name  Category    Rating  \
0             Smoke Effect Photo Maker - Smoke Editor       0.0  3.063063   
1                                    Infinite Painter       0.0  4.041739   
2                                Garden Coloring Book       0.0  4.339952   
3                      Tattoo Name On My Photo Editor       0.0  4.120000   
4                      Tattoo Name On My Photo Editor       0.0  4.083053   
5                      Tattoo Name On My Photo Editor       0.0  4.120000   
6                               Mandala Coloring Book       0.0  0.000000   
7                               Mandala Coloring Book       0.0  0.000000   
8                               Mandala Coloring Book       0.0  0.000000   
9                               Mandala Coloring Book       0.0  3.750000   
10                              Mandala Coloring Book       0.0  4.028568   
11                              Mandala Coloring Book       0.0  4.383333   

In [ ]:
from sklearn.preprocessing import MinMaxScaler

numeric_columns = [
        'Rating', 'Rating Count', 'Minimum Installs', 'Price', 'Reviews'
    ]

scaler = MinMaxScaler(feature_range=(0, 1))
for col in numeric_columns:
  normalized_data[col] = scaler.fit_transform(merged_data[[col]])

print(normalized_data.head(50))

                                             App Name  Category    Rating  \
0             Smoke Effect Photo Maker - Smoke Editor       0.0  0.612613   
1                                    Infinite Painter       0.0  0.808348   
2                                Garden Coloring Book       0.0  0.867990   
3                      Tattoo Name On My Photo Editor       0.0  0.824000   
4                      Tattoo Name On My Photo Editor       0.0  0.816611   
5                      Tattoo Name On My Photo Editor       0.0  0.824000   
6                               Mandala Coloring Book       0.0  0.000000   
7                               Mandala Coloring Book       0.0  0.000000   
8                               Mandala Coloring Book       0.0  0.000000   
9                               Mandala Coloring Book       0.0  0.750000   
10                              Mandala Coloring Book       0.0  0.805714   
11                              Mandala Coloring Book       0.0  0.876667   

In [ ]:
outlier_removal_columns = ['Reviews', 'Rating', 'Rating Count']
print(len(merged_data))
for column in outlier_removal_columns:
  Q1 = merged_data[column].quantile(0.25)
  Q3 = merged_data[column].quantile(0.75)
  IQR = Q3 - Q1
  for index, value in enumerate(merged_data[column]):
    if (value < (Q1 - 1.5 * IQR)) | (value > (Q3 + 1.5 * IQR)):
      merged_data = merged_data.loc[merged_data.index != index]
print(len(merged_data))

6250
4182


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
nltk_columns = ['Developer Address', 'Summary']

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def normalize_text(text):
  tokens = nltk.word_tokenize(text)
  lemmas = [lemmatizer.lemmatize(token) for token in tokens]
  tokens = [token for token in lemmas if token not in stop_words]
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  return stemmed_tokens


for col in nltk_columns:
  merged_data[col] = merged_data[col].astype(str)
  merged_data['normalized_' + col] = merged_data[col].apply(normalize_text)

print(merged_data['normalized_Summary'].head(50))

0     [prank, parent, friend, smoke, effect, photo, ...
1                        [paint, ,, draw, ,, &, sketch]
2     [a, magic, mind, journey, colorfit, garden, co...
3     [thi, perfect, tattoo, maker, app, thousand, t...
4     [tattoo, photo, name, tattoo, design, app, gir...
5     [thi, perfect, tattoo, maker, app, thousand, t...
9         [beauti, mandala, color, page, adult, kid, .]
10    [mandala, color, book, open, mysteri, tour, na...
11    [best, mandala, color, book, adult, !, color, ...
12    [play, relax, 1000+, 3d, pixel, art, lot, free...
13                 [there, 350, diy, room, decor, idea]
15    [1000+page, :, draw, creation, mandala, ., anx...
16    [contain, collect, best, draw, art, design, id...
17                                    [art, draw, idea]
18     [free, easi, origami, app, step-by-step, manual]
19    [sandbox, pixel, color, book, &, color, by, nu...
20    [sandbox, pixel, color, book, &, color, by, nu...
22          [floor, plan, creator, ,, fast, free

In [ ]:
ratings_mean = merged_data['Rating'].mean()
print(ratings_mean)
sport_rating = merged_data[merged_data['Genres'] == 'Sports']['Rating']

print(sport_rating - ratings_mean)

3.7073452766140607
718     0.057361
720     0.057361
3105    0.900443
3106    0.900443
3107    0.900443
          ...   
6173    1.122006
6174    0.126217
6186    0.491450
6205    0.269991
6215    0.301186
Name: Rating, Length: 101, dtype: float64


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
numeric_columns = [
        'Rating', 'Rating Count', 'Minimum Installs', 'Price', 'Reviews', 'Category', 'Type', 'Content Rating', 'Genres', 'Android Ver', 'Currency', 'Minimum Android', 'Ad Supported', 'In app purchases', 'Editor Choice'
  ]
plt.figure(figsize=(8, 6))
for i in range(len(numeric_columns) - 1):
  for j in range(i + 1, len(numeric_columns)):
    feature1 = numeric_columns[i]
    feature2 = numeric_columns[j]
    plt.scatter(normalized_data[feature1], normalized_data[feature2])
    plt.title('Scatter Plot of ' + feature1 + ' vs ' + feature2)
    plt.xlabel(feature1)
    plt.ylabel(feature2)
    plt.grid(True)
    plt.show()

In [ ]:
merged_data.head(10)

,App Name,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Summary,Developer,Developer Address,Developer Internal ID,price_currency,rating_and_count,name_and_category,installs_and_editor_choice,normalized_Developer Address,normalized_Summary
0,Smoke Effect Photo Maker - Smoke Editor,Art & Design,3.063063,448.0,19M,"100,000+",Free,0.0,Everyone,Art & Design,...,Prank your Parents or Friends with Smoke Effec...,Anzu Dev,nan,7.028150e+18,0.0 USD,3.0630631 - 1114.0,Smoke Effect Photo Maker - Smoke Editor - Art ...,"100,000+ - False",[nan],"[prank, parent, friend, smoke, effect, photo, ..."
1,Infinite Painter,Art & Design,4.041739,42184.0,124M,"10,000,000+",Free,0.0,Everyone,Art & Design,...,"Painting, drawing, & sketching",Infinite Studio LLC,1155 SEMINOLE TRAIL\nPO BOX 6402\nCHARLOTTESVI...,8.161090e+18,0.0 USD,4.0417385 - 92790.0,Infinite Painter - Art & Design,"10,000,000+ - False","[1155, seminol, trail, po, box, 6402, charlott...","[paint, ,, draw, ,, &, sketch]"
2,Garden Coloring Book,Art & Design,4.339952,5589.0,39M,"1,000,000+",Free,0.0,Teen,Art & Design,...,A magical mind journey in Colorfit Garden Colo...,Colorfit,Flat 11 1/F Block 10 Shek Lei (2) Estate ...,7.021960e+18,0.0 USD,4.339952 - 16599.0,Garden Coloring Book - Art & Design,"1,000,000+ - False","[flat, 11, 1/f, block, 10, shek, lei, (, 2, ),...","[a, magic, mind, journey, colorfit, garden, co..."
3,Tattoo Name On My Photo Editor,Art & Design,4.120000,41.0,30M,"10,000+",Free,0.0,Teen,Art & Design,...,This is the perfect tattoo maker app with thou...,Voodamdee,nan,8.883360e+18,0.0 USD,4.12 - 110.0,Tattoo Name On My Photo Editor - Art & Design,"10,000+ - False",[nan],"[thi, perfect, tattoo, maker, app, thousand, t..."
4,Tattoo Name On My Photo Editor,Art & Design,4.083053,20280.0,23M,"10,000,000+",Free,0.0,Teen,Art & Design,...,Tattoo my photo with my name is tattoo design ...,iDroid Solution,"Surat, Gujarat, India",5.388640e+18,0.0 USD,4.0830526 - 50147.0,Tattoo Name On My Photo Editor - Art & Design,"10,000,000+ - False","[surat, ,, gujarat, ,, india]","[tattoo, photo, name, tattoo, design, app, gir..."
5,Tattoo Name On My Photo Editor,Art & Design,4.120000,41.0,30M,"10,000+",Free,0.0,Teen,Art & Design,...,This is the perfect tattoo maker app with thou...,Voodamdee,nan,8.883360e+18,0.0 USD,4.12 - 110.0,Tattoo Name On My Photo Editor - Art & Design,"10,000+ - False",[nan],"[thi, perfect, tattoo, maker, app, thousand, t..."
9,Mandala Coloring Book,Entertainment,3.750000,9.0,32M,"5,000+",Free,0.0,Everyone,Entertainment,...,Beautiful mandala coloring pages for adults an...,ArtiomApps,nan,5.595870e+18,0.0 USD,3.75 - 16.0,Mandala Coloring Book - Entertainment,"5,000+ - False",[nan],"[beauti, mandala, color, page, adult, kid, .]"
10,Mandala Coloring Book,Casual,4.028568,28588.0,37M,"10,000,000+",Free,0.0,Everyone,Casual,...,Mandala Coloring Book opens a mysterious tour ...,Colorfit,Flat 11 1/F Block 10 Shek Lei (2) Estate ...,7.021960e+18,0.0 USD,4.028568 - 76890.0,Mandala Coloring Book - Casual,"10,000,000+ - False","[flat, 11, 1/f, block, 10, shek, lei, (, 2, ),...","[mandala, color, book, open, mysteri, tour, na..."
11,Mandala Coloring Book,Art & Design,4.383333,2040.0,38M,"500,000+",Free,0.0,Everyone,Art & Design,...,Best Mandala coloring book for adults! Color m...,Piu Piu Apps,"4a Inzhenernaya str., door 420\nNovosibirsk, 6...",8.457480e+18,0.0 USD,4.383333 - 4200.0,Mandala Coloring Book - Art & Design,"500,000+ - False","[4a, inzhenernaya, str., ,, door, 420, novosib...","[best, mandala, color, book, adult, !, color, ..."
12,3D Color Pixel by Number - Sandbox Art Coloring,Entertainment,3.972568,2032.0,Varies with device,"500,000+",Free,0.0,Everyone,Entertainment,...,Play and relax with 1000+ 3D pixel art with lo...,Touchzing Media Private Limited,"607, Western Edge II, Mumbai, India",5.012320e+18,0.0 USD,3.9725685 - 4005.0,3D Color Pixel by Number - Sandbox Art Colorin...,"500,000+ - False","[607, ,, western, edg, ii, ,, mumbai, ,, india]

In [ ]:
for col in normalized_data.columns:
  print(col)
  print(normalized_data[col].head(10))
  print('---------------------------')
  print()

App Name
0    Smoke Effect Photo Maker - Smoke Editor
1                           Infinite Painter
2                       Garden Coloring Book
3             Tattoo Name On My Photo Editor
4             Tattoo Name On My Photo Editor
5             Tattoo Name On My Photo Editor
6                      Mandala Coloring Book
7                      Mandala Coloring Book
8                      Mandala Coloring Book
9                      Mandala Coloring Book
Name: App Name, dtype: object
---------------------------

Category
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: Category, dtype: float64
---------------------------

Rating
0    0.612613
1    0.808348
2    0.867990
3    0.824000
4    0.816611
5    0.824000
6    0.000000
7    0.000000
8    0.000000
9    0.750000
Name: Rating, dtype: float64
---------------------------

Reviews
0    1.078849e-05
1    1.015852e-03
2    1.345913e-04
3    9.873399e-07
4    4.883720e-04
5    9.873399e-07
6 

In [ ]:
columns_to_delete = ['App Name', 'Last Updated', 'installs_and_editor_choice', 'name_and_category', 'rating_and_count', 'price_currency','Current Ver','Android Ver',
                     'App Id','Minimum Android','Developer Id','Developer Website','Developer Email','Released','Privacy Policy','Summary','Developer',
                     'Developer','Developer Address','Developer Internal ID']

clean_data = normalized_data.drop(columns=columns_to_delete)

In [ ]:
clean_data['Installs'] = clean_data['Installs'].str.replace('+', '')
clean_data['Installs'] = clean_data['Installs'].str.replace(',', '')
clean_data['Installs'] = pd.to_numeric(clean_data['Installs'], errors='coerce').astype('Int64')

clean_data['Size'] = clean_data['Size'].str.replace('M', '')
clean_data['Size'] = clean_data['Size'].str.replace('Varies with device', '')
clean_data['Size'] = pd.to_numeric(clean_data['Size'], errors='coerce')

In [ ]:
csv_file_path = 'clean_dataset.csv'
clean_data.to_csv(csv_file_path, index=False)

In [ ]:
def find_columns_with_two_unique_values(df):
    cols_with_two_unique = [col for col in df.columns if df[col].nunique() == 2 and col != 'Genres' and col != 'Category']
    return cols_with_two_unique

# Find columns with exactly 2 unique values
columns_with_two_unique_values = find_columns_with_two_unique_values(clean_data)

# Display the columns
print("Columns with exactly 2 unique values:")
print(columns_with_two_unique_values)

Columns with exactly 2 unique values:
['Type', 'Content Rating', 'Currency', 'In app purchases', 'Editor Choice']


In [ ]:
selected_data = clean_data[columns_with_two_unique_values]

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(selected_data, min_support=0.2, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

print("Frequent Itemsets:")
print(frequent_itemsets)

print("\nAssociation Rules:")
print(rules)

frequent_patterns = frequent_itemsets[frequent_itemsets['support'] > 0.2].head(7)
print("\nFrequent Patterns (at least 7):")
print(frequent_patterns)

Frequent Itemsets:
    support                                           itemsets
0   0.93584                                             (Type)
1   0.84240                                   (Content Rating)
2   0.64864                                 (In app purchases)
3   0.95840                                    (Editor Choice)
4   0.78784                             (Content Rating, Type)
5   0.59856                           (In app purchases, Type)
6   0.89568                              (Editor Choice, Type)
7   0.57904                 (In app purchases, Content Rating)
8   0.81888                    (Editor Choice, Content Rating)
9   0.64016                  (Editor Choice, In app purchases)
10  0.53488           (Content Rating, In app purchases, Type)
11  0.76480              (Editor Choice, Content Rating, Type)
12  0.59056            (Editor Choice, In app purchases, Type)
13  0.57328  (Editor Choice, In app purchases, Content Rating)
14  0.52928  (Editor Choice, Content